In [1]:
#Import libraries
import pandas as pd
import numpy as np
import pypyodbc as podbc
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV, LassoCV,Ridge,Lasso
import seaborn as sns
from sklearn.feature_selection import SelectFromModel

In [2]:
#pandas params
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
#Connect to RDS db
conn = podbc.connect(
    #Trusted_Connection='Yes',
    Driver='{SQL Server}',
    Server='nbahistorical.cmgzaupuq9dz.us-east-2.rds.amazonaws.com',
    UID='admin',
    PWD='Gs239350',
    Database='NBA_historical',
    MultipleActiveResultSets = 'True'
)

In [4]:
#Query data
data = pd.read_sql(sql = "SELECT * FROM box_scores WHERE CAST(SUBSTRING(SEASON_ID,1,4) AS int)>=1984 AND TOV IS NOT NULL;",con = conn)

In [5]:
#plt.figure(figsize=(12,10))
#cor = data.corr()
#sns.heatmap(cor, annot = False, cmap = plt.cm.Reds)

In [6]:
data.insert(loc = 37, column = 'reb_ast', value = (data['reb']/data['ast']))
data.insert(loc = 37, column = 'ftpercent', value = (data['ftm']/data['fta']))
data.insert(loc = 37, column = '3ptrate', value = (data['fg3a']/data['fga']))
data.insert(loc = 37, column = 'fga_ast', value = (data['fga']/data['ast']))
data.insert(loc = 37, column = 'reb_point', value = (data['reb']/data['pts']))
data.insert(loc = 37, column = 'fta_fg2a', value = (data['fta']/data['fg2a']))
data.insert(loc = 37, column = 'fg3a_reb', value = (data['fg3a']/data['reb']))
data.insert(loc = 37, column = 'fga_plus_ast', value = (data['fga']+data['ast']))
data.insert(loc = 37, column = 'usg', value = (data['fga']+(data['fta']*0.44)-data['oreb']))

In [7]:
data = data.replace([np.inf, -np.inf], np.nan).assign()

In [8]:
#Split data into testing set and training set
train = data.iloc[0:459146]
test = data.iloc[459147:]

In [9]:
train = train.fillna(0).assign()
train.isnull().sum()

game_id              0
team_id              0
team_abbreviation    0
team_city            0
player_name          0
player_id            0
start_position       0
comment              0
min                  0
fgm                  0
fga                  0
fg3m                 0
fg3a                 0
ftm                  0
fta                  0
oreb                 0
dreb                 0
reb                  0
ast                  0
stl                  0
blk                  0
tov                  0
pf                   0
pts                  0
plus_minus           0
misses               0
ft_misses            0
fg2m                 0
fg2a                 0
fg2percent           0
fg3percent           0
fgpercent            0
efg                  0
ts                   0
season_id            0
minutes              0
unique_team          0
usg                  0
fga_plus_ast         0
fg3a_reb             0
fta_fg2a             0
reb_point            0
fga_ast              0
3ptrate    

In [11]:
X = train[['misses','ast','minutes']].copy()
target = pd.DataFrame(data = train['tov'])
y = target['tov']

#'reb','misses','ft_misses','minutes','efg','fg3m','fg3a','reb_ast',

model = sm.OLS(y.astype(float), X.astype(float)).fit()
predictions = model.predict(X) 

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    tov   R-squared (uncentered):                   0.631
Model:                            OLS   Adj. R-squared (uncentered):              0.631
Method:                 Least Squares   F-statistic:                          2.621e+05
Date:                Mon, 10 Aug 2020   Prob (F-statistic):                        0.00
Time:                        00:15:10   Log-Likelihood:                     -7.6843e+05
No. Observations:              459146   AIC:                                  1.537e+06
Df Residuals:                  459143   BIC:                                  1.537e+06
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
misses         0.0223      0.001     28.118      0.000       0.021       0.024
ast            0.0686      0.001     83.737      0.000       0.067       0.070
minutes        0.0518      0.000    284.091      0.000       0.051       0.052
==============================================================================
Omnibus:                    59385.343   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           105370.188
Skew:                           0.861   Prob(JB):                         0.00
Kurtosis:                       4.596   Cond. No.                         11.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""